# Кластеризация текстов

In [25]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.datasets import fetch_20newsgroups

# Посмотрим на темы текстов из стандартного датасета, которыми можно воспользоваться,
# чтоб проверить качество работы алгоритма
train_all = fetch_20newsgroups(subset='train')
print(train_all.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [3]:
# Взяли тексты с тремя абсолютно разными темами.
simple_dataset = fetch_20newsgroups(
        subset='train',
        categories=['comp.sys.mac.hardware', 'soc.religion.christian', 'rec.sport.hockey'])

In [4]:
print(simple_dataset.data[0]) # первый элемент

From: erik@cheshire.oxy.edu (Erik Adams)
Subject: HELP!!  My Macintosh "luggable" has lines on its screen!
Organization: Occidental College, Los Angeles, CA 90041 USA.
Distribution: comp
Lines: 20

Okay, I don't use it very much, but I would like for it to keep working
correctly, at least as long as Apple continues to make System software
that will run on it, if slowly :-)

Here is the problem:  When the screen is tilted too far back, vertical
lines appear on the screen.  They are every 10 pixels or so, and seem
to be affected somewhat by opening windows and pulling down menus.
It looks to a semi-technical person like there is a loose connection
between the screen and the rest of the computer.

I am open to suggestions that do not involve buying a new computer,
or taking this one to the shop.  I would also like to not have
to buy one of Larry Pina's books.  I like Larry, but I'm not sure
I feel strongly enough about the computer to buy a service manual
for it.

On a related note:  what

In [5]:
simple_dataset.target # метки классов

array([0, 0, 1, ..., 0, 1, 2])

In [6]:
print(simple_dataset.data[-1]) # последний из текстов

From: dlecoint@garnet.acns.fsu.edu (Darius_Lecointe)
Subject: Re: Sabbath Admissions 5of5
Organization: Florida State University
Lines: 21

I find it interesting that cls never answered any of the questions posed. 
Then he goes on the make statements which make me shudder.  He has
established a two-tiered God.  One set of rules for the Jews (his people)
and another set for the saved Gentiles (his people).  Why would God
discriminate?  Does the Jew who accepts Jesus now have to live under the
Gentile rules.

God has one set of rules for all his people.  Paul was never against the
law.  In fact he says repeatedly that faith establishes rather that annuls
the law.  Paul's point is germane to both Jews and Greeks.  The Law can
never be used as an instrument of salvation.  And please do not combine
the ceremonial and moral laws in one.

In Matt 5:14-19 Christ plainly says what He came to do and you say He was
only saying that for the Jews's benefit.  Your Christ must be a
politician, speaki

In [9]:
print(len(simple_dataset.data))   # количество объектов в выборке

1777


### Призаки

Вычислим частоты слов.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [26]:
vectorizer = TfidfVectorizer(max_df=500, min_df=100) # задали максимальную и минимальную документную частоту

"""Не хотим видеть слова, которые встречаются более чем в 500 документах и менее чем в 10, чтобы количество кластеров
было ограничено"""
matrix = vectorizer.fit_transform(simple_dataset.data)
print(matrix.shape) # 3767 частот на 1777 объектах

(1777, 333)


### Метод $k$-средних

In [28]:
from sklearn.cluster import KMeans

In [29]:
model = KMeans(n_clusters=3, random_state=42)
preds = model.fit_predict(matrix.toarray())
print(preds) # прогнозы

[1 1 0 ... 1 0 2]


In [30]:
print(simple_dataset.target) # посмотрим, которые ответы должны были получится

[0 0 1 ... 0 1 2]


In [42]:
# Не угадали с тем, как пометить кластер. Сделаем отображение, чтобы двойка перешла в единицу, а единица - в двойку
"""
Прим. моё: вроде как тут и в последующих местах должно быть другое соответствие. 
Также интересен факт, что в видео указан не вероятность правильных угадывания, а вероятность ошибок. 
Нигде в .ipynb я не стал делать намеренных расхождений с видео, однако они могут быть нужны!
"""
mapping = {2: 1, 1: 2, 0: 0}
mapped_preds = [mapping[pred] for pred in preds]

"""КЛАСТЕРИЗАЦИЯ"""
print('\nУгадываем кластер с вероятностью: ')
print(float(sum(mapped_preds != simple_dataset.target)) / len(simple_dataset.target))


Угадываем кластер с вероятностью: 
0.9296567248171075


In [44]:
# Сравним ответы с классификатором, который знает ответы, которые должны получиться

from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
clf = LogisticRegression()

"""КЛАССИФИКАЦИЯ"""
print('\nОтветы при классификации (обучении с учителем): ')
print(cross_val_score(clf, matrix, simple_dataset.target).mean())


Ответы при классификации (обучении с учителем): 
0.9515973631072799


## Более сложная выборка

Попробуем взять выборку текстов только про компьютеры, но с тремя *под*классами: hardware, misc, graphics

In [47]:
dataset = fetch_20newsgroups(
        subset='train',
        categories=['comp.sys.mac.hardware', 'comp.os.ms-windows.misc', 'comp.graphics'])

Попробуем сразу для кластеризации применим метод $k$-средних:

In [49]:
matrix = vectorizer.fit_transform(dataset.data)
model = KMeans(n_clusters=3, random_state=42)
preds = model.fit_predict(matrix.toarray())

print('\nПредсказания при кластеризации: ', preds)
print('Ответы при классификации (обучении с учителем): ', dataset.target)


Предсказания при кластеризации:  [0 0 1 ... 0 1 0]
Ответы при классификации (обучении с учителем):  [2 1 1 ... 2 0 2]


In [53]:
mapping = {2: 0, 1: 1, 0: 2} # сопоставляем полученные кластеры с правильными ответами
mapped_preds = [mapping[pred] for pred in preds]

"""КЛАСТЕРИЗАЦИЯ"""
print('\nУгадываем кластер с вероятностью: ')
print(float(sum(mapped_preds != dataset.target)) / len(dataset.target))


Угадываем кластер с вероятностью: 
0.6103822019395322


In [54]:
clf = LogisticRegression()

"""КЛАССИФИКАЦИЯ"""
print('\nОтветы при классификации (обучении с учителем): ')
print(cross_val_score(clf, matrix, dataset.target).mean())


Ответы при классификации (обучении с учителем): 
0.8014748354370996


### Уменьшим количество признаков сингулярным разложением (SVD)

In [61]:
from sklearn.decomposition import TruncatedSVD

model = KMeans(n_clusters=3, random_state=42)
svd = TruncatedSVD(n_components=249, random_state=123) # уменьшим количество признаков до 249
features = svd.fit_transform(matrix)
preds = model.fit_predict(features)

print('\nПредсказания при кластеризации: ', preds)
print('Ответы при классификации (обучении с учителем): ', dataset.target)


Предсказания при кластеризации:  [1 1 0 ... 1 0 1]
Ответы при классификации (обучении с учителем):  [2 1 1 ... 2 0 2]


In [63]:
mapping = {2: 1, 1: 0, 0: 2} # сопоставляем полученные кластеры с правильными ответами
mapped_preds = [mapping[pred] for pred in preds]

"""КЛАСТЕРИЗАЦИЯ"""
print('\nУгадываем кластер с вероятностью: ')
print(float(sum(mapped_preds != dataset.target)) / len(dataset.target))


Угадываем кластер с вероятностью: 
0.6668568168853394


In [64]:
clf = LogisticRegression()

"""КЛАССИФИКАЦИЯ"""
print('\nОтветы при классификации (обучении с учителем): ')
print(cross_val_score(clf, matrix, dataset.target).mean())


Ответы при классификации (обучении с учителем): 
0.8014748354370996
